In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from pathlib import Path
import sys

# Add the parent directory to the Python path
script_dir = Path().resolve() # Get the current working directory
sys.path.append(str(script_dir.parent))
import src.config as config 

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/840852
2024-07-08 00:03:14,613 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-07-08 00:03:14,613 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/840852/fs/836675/fv/time_series_hourly_feature_view/version/1


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (52.60s) 
2024-07-08 01:28:51,242 WARNING: VersionWarning: Incremented version to `1`.



In [6]:
# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1538787,2022-01-01 00:00:00+00:00,0,1
2081971,2022-01-01 01:00:00+00:00,0,1
4165882,2022-01-01 02:00:00+00:00,0,1
229884,2022-01-01 03:00:00+00:00,0,1
4832274,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
3383324,2024-07-07 17:00:00+00:00,3,265
3383125,2024-07-07 18:00:00+00:00,3,265
3383036,2024-07-07 19:00:00+00:00,3,265
3384362,2024-07-07 20:00:00+00:00,7,265


In [7]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [03:32<00:00,  1.25it/s]


features_and_target.shape=(219776, 675)


In [8]:
features_and_target.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-01-29 23:00:00+00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,2.0,0.0,1.0,2.0,0.0,0.0,2022-01-30 22:00:00+00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,1.0,2022-01-31 21:00:00+00:00,1,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-02-01 20:00:00+00:00,1,0.0


In [10]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# Define the cutoff date for splitting the data
# Training data -> from January 2022 up until 2 months ago
# Test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*2), utc=True)  # Use 28*2 to approximate 2 months

print(f'{cutoff_date=}')

# Ensure the 'pickup_hour' column in 'features_and_target' is in datetime format
features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour'], utc=True)

# Split the data into training and test sets
X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

# Print the shapes of the resulting datasets
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')


cutoff_date=Timestamp('2024-05-13 00:00:00+0000', tz='UTC')
X_train.shape=(210257, 674)
y_train.shape=(210257,)
X_test.shape=(9519, 674)
y_test.shape=(9519,)


In [11]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    X_train.sort_values('pickup_hour', inplace=True)

    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2024-07-08 02:01:33,072] A new study created in memory with name: no-name-c3b7d16f-19bf-4425-aa07-4d2f60158250
[I 2024-07-08 02:03:42,437] Trial 0 finished with value: 28.1363148244824 and parameters: {'num_leaves': 157, 'feature_fraction': 0.671959275403464, 'bagging_fraction': 0.5852917501207565, 'min_child_samples': 89}. Best is trial 0 with value: 28.1363148244824.


In [14]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 157, 'feature_fraction': 0.671959275403464, 'bagging_fraction': 0.5852917501207565, 'min_child_samples': 89}


In [15]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.671959275403464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.671959275403464
[LightGBM] [Warning] bagging_fraction is set=0.5852917501207565, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5852917501207565
[LightGBM] [Warning] feature_fraction is set=0.671959275403464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.671959275403464
[LightGBM] [Warning] bagging_fraction is set=0.5852917501207565, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5852917501207565
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.197900 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171893
[LightGBM] [Info] Number of data points in the train set: 210257, number of used features: 676
[LightGBM] [Info] Start training from score 16.745207


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000002268763A020>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5852917501207565,
                               feature_fraction=0.671959275403464,
                               min_child_samples=89, num_leaves=157))])

In [17]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.671959275403464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.671959275403464
[LightGBM] [Warning] bagging_fraction is set=0.5852917501207565, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5852917501207565
test_mae=22.1059


In [18]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['D:\\TAXI_DEMAND_PREDICTOR\\taxi_demand_predictor\\models\\model.pkl']

In [19]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [21]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model_save_path = str(MODELS_DIR / 'model.pkl')
model.save(model_save_path)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1472262 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3399 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/840852/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)